In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.applications.vgg19 import VGG19
from keras.models import load_model
import tensorflow as tf
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd

Using TensorFlow backend.


In [14]:
use_log_dir='X:/train-scene classification/logs_vgg_retraining_with_more_augumentation' # Change for every new training session 

class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir=use_log_dir, **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()

In [3]:
img_width, img_height = 150, 150
train_data_dir = "X:/train-scene classification/data/train/"
validation_data_dir = "X:/train-scene classification/data/valid/"
test_data_dir = "X:/train-scene classification/data/test/"
nb_train_samples = 16384
nb_validation_samples = 650 
batch_size = 32
epochs = 75
# Save the model according to the conditions  
chkpt_dir = "X:/train-scene classification/chkpts/vgg19_retraining_with_more_augumentation.h5" # Change for every new training session 

In [4]:
# model = VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3), classes = 6)

In [5]:
# model.save('vgg19.h5')
model = load_model('vgg19.h5')

C:\Users\praty\Anaconda3\envs\dlenv\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
# for layer in model.layers[:200]:
#     layer.trainable = False

model.summary()
print(len(model.layers))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [7]:
#Adding custom Layers 
x = model.output

# x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(6, activation="softmax")(x)

In [8]:
# creating the final model 
model_final = Model(input = model.input, output = predictions)
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

C:\Users\praty\Anaconda3\envs\dlenv\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [9]:
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8), metrics=["accuracy"])

In [10]:
model_final.load_weights(chkpt_dir)

OSError: Unable to open file (unable to open file: name = 'X:/train-scene classification/chkpts/vgg19_retraining_with_more_augumentation.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [11]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
shear_range = 10,
fill_mode = "nearest",
zoom_range = 0.4,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=10)

validation_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
shear_range = 10,
fill_mode = "nearest",
zoom_range = 0.4,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=10)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = validation_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

Found 15000 images belonging to 6 classes.
Found 2034 images belonging to 6 classes.


In [15]:
# Save the model according to the conditions  
# chkpt_dir = "X:/train-scene classification/chkpts/densenet_extra_fc.h5" # Change for every new training session 

checkpoint = ModelCheckpoint(chkpt_dir, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=40, verbose=1, mode='auto')
#[TrainValTensorBoard(write_graph=False)]

In [16]:
import os

# Train the model 
model_final.fit_generator(
train_generator,
samples_per_epoch = nb_train_samples,
epochs = epochs,
validation_data = validation_generator,
nb_val_samples = nb_validation_samples,
callbacks = [checkpoint, early, TrainValTensorBoard(write_graph=True)])

C:\Users\praty\Anaconda3\envs\dlenv\lib\site-packages\ipykernel_launcher.py:10: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\praty\Anaconda3\envs\dlenv\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=75, validation_data=<keras_pre..., callbacks=[<keras.ca..., steps_per_epoch=512, validation_steps=650)`
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/75
431/512 [========================>.....] - ETA: 18s - loss: 1.8073 - acc: 0.1765

KeyboardInterrupt: 

In [ ]:
#Saving Model
model_final.save("vgg_retraining_with_more_augumentation.h5")

## Testing + Predictions

In [ ]:
model_final = load_model(chkpt_dir)

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
test_data_dir_2 = "X:/train-scene classification/data/test/final_test"

In [ ]:
labels = (train_generator.class_indices)

labels = dict((v,k) for k,v in labels.items())
print(labels)
# predictions = [labels[k] for k in predicted_class_indices]

In [ ]:
# train_dir = "X:/train-scene classification/data/train/1_forest"
# img = cv2.imread(os.path.join(train_dir, "466.jpg"))

# valid_dir = "X:/train-scene classification/data/valid/1_forest"
# img = cv2.imread(os.path.join(valid_dir, "2800.jpg"))

img = cv2.imread(os.path.join(test_data_dir_2, "398.jpg"))
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img_rgb)
print(img.shape)
img = np.reshape(img, (1, img_width, img_height, 3))
pred = np.argmax(model_final.predict(img), axis=1)
print(pred, labels[pred[0]])

In [ ]:
#### model = load_model("")

In [ ]:
# test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# test_generator = test_datagen.flow_from_directory(
#     directory="X:/train-scene classification/data/test/",
#     target_size=(img_width, img_height),
#     color_mode="rgb",
#     batch_size=1,
#     class_mode=None,
#     shuffle=False,
#     seed=42
# )

In [ ]:
# test_generator.reset()
# pred=model_final.predict_generator(test_generator,verbose=1)

In [ ]:
# import numpy as np
# predicted_class_indices=np.argmax(pred,axis=1)

In [ ]:
# print(predicted_class_indices[:20])
# print(predicted_class_indices.shape)
# print(pred.shape)

In [ ]:
import pandas as pd
df = pd.read_csv(os.path.join("X:/train-scene classification", 'test_images.csv'))
print(df.head())
test_list = df.image_name.values
print(test_list[:20])

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
join_l = []
ans = []
test_data_dir_2 = "X:/train-scene classification/data/test/final_test"
for i, im in enumerate(test_list):
    img = cv2.imread(os.path.join(test_data_dir_2, im))
    if i == 1:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img_rgb)
    img = cv2.resize(img, (img_width, img_height)) 
    img = np.reshape(img, (1, img_width, img_height, 3))
    img = img / 255.
    pred = np.argmax(model_final.predict(img), axis=1)
    ans.append((im, pred))

# x_test = np.vstack(join_l)
# print(x_test.shape)

In [ ]:
pred = model_final.predict(x_test)

In [ ]:
print(pred[:10])

In [ ]:
preds = np.argmax(pred, axis=1)

In [ ]:
print(preds)

In [ ]:
print(ans[0:10])

In [ ]:
print(ans[0][1].shape)

In [ ]:
li = []
for i in ans:
    pa = i[0]
    val= i[1][0]
    li.append([pa, val])
print(li[:10])

In [ ]:
res_df = pd.DataFrame(li, columns=["image_name", "label"])
res_df.head()

In [ ]:
res_df.to_csv('submission_vgg19_normalization_corrected.csv', index = False)